Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. 

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:
- логистическая регрессия по признакам $X_1,X_2,X_3$
- логистическая регрессия по признакам $X_4,X_5,X_6$

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости.

In [ ]:
!pip install -q 'statsmodels==0.12.2'

# Z-критерий для двух долей 

In [ ]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Загрузка данных

In [ ]:
path = 'https://raw.githubusercontent.com/chekhovana/courses/main/machine_learning/4_inference/2_%20ab_testing/data/2.4.2_banknotes.txt'
data = pd.read_csv(path, sep = '\t')
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [ ]:
X = data.loc[:, data.columns != 'real']
X.head()
y = data['real']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
columns = ['X1', 'X2', 'X3']
lr1 = LogisticRegression(solver='liblinear', random_state=1)
lr1.fit(X_train[columns], y_train)
y_pred1 = lr1.predict(X_test[columns])
errors1 = abs(y_pred1 - y_test)
print(sum(errors1))

10


In [ ]:
columns = ['X4', 'X5', 'X6']
lr2 = LogisticRegression(solver='liblinear', random_state=1)
lr2.fit(X_train[columns], y_train)
y_pred2 = lr2.predict(X_test[columns])
errors2 = abs(y_pred2 - y_test)
print(sum(errors2))

1


## Z-критерий для разности долей (связанные выборки)

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$

In [ ]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [ ]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [ ]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [ ]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(errors2, errors1), 'less'))

p-value: 0.001648


Посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов.

In [ ]:
proportions_diff_confint_rel(errors1, errors2)

(0.059945206279614305, 0.3000547937203857)

In [ ]:
from statsmodels.stats.proportion import test_proportions_2indep
from statsmodels.stats.proportion import confint_proportions_2indep

test_proportions_2indep(sum(errors1), len(errors1), sum(errors2), len(errors2), alternative='larger', method='wald')
confint_proportions_2indep(sum(errors1), len(errors1), sum(errors2), len(errors2))

(0.05799816576340375, 0.3114061227421797)